# Project 6: Analyzing Stock Sentiment from Twits
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

### Load Packages

In [1]:
import json
import nltk
import os
import random
import re
import torch
import numpy as np

from torch import nn, optim
import torch.nn.functional as F

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [2]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [3]:
"""print out the number of twits"""

# TODO Implement 
print("Number of twits: {}".format(len(twits["data"])))

Number of twits: 1548010


### Split Message Body and Sentiment Score

In [4]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

In [5]:
messages

['$FITB great buy at 26.00...ill wait',
 '@StockTwits $MSFT',
 '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating',
 '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.',
 '$AMD reveal yourself!',
 '$AAPL Why the drop? I warren Buffet taking out his position?',
 '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA',
 '$BAC ok good we&#39;re not dropping in price over the weekend, lol',
 '$AMAT - Daily Chart, we need to get back to above 50.',
 '$GME 3% drop per week after spike... if no news in 3 months, back to 12s... if BO, then bingo... what is the odds?',
 '$SBUX STRONG BUY!',
 '$SNPS short ratio is 2.17 at 2018-06-15 and short % to float is 1.42% http://sunshineavenue.com/stock/SNPS/ via @sunshineave',
 '$NFLX price squeezing,perfect place for an option straddle near the supporting

In [6]:
sentiments

[4,
 3,
 4,
 3,
 2,
 3,
 0,
 3,
 4,
 0,
 4,
 0,
 4,
 4,
 4,
 3,
 4,
 2,
 4,
 3,
 3,
 3,
 0,
 1,
 2,
 3,
 3,
 0,
 4,
 2,
 4,
 0,
 3,
 2,
 2,
 1,
 2,
 2,
 3,
 2,
 3,
 4,
 3,
 2,
 2,
 2,
 2,
 4,
 3,
 3,
 2,
 2,
 3,
 3,
 3,
 3,
 1,
 4,
 3,
 4,
 0,
 3,
 2,
 0,
 2,
 1,
 4,
 1,
 3,
 2,
 4,
 1,
 2,
 0,
 2,
 2,
 4,
 2,
 2,
 2,
 2,
 2,
 4,
 2,
 4,
 2,
 3,
 4,
 2,
 0,
 2,
 2,
 0,
 0,
 2,
 3,
 1,
 1,
 2,
 2,
 2,
 2,
 3,
 2,
 0,
 2,
 0,
 4,
 4,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 3,
 2,
 4,
 0,
 4,
 3,
 3,
 2,
 2,
 4,
 2,
 2,
 4,
 2,
 2,
 1,
 1,
 4,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 4,
 1,
 2,
 2,
 2,
 0,
 3,
 0,
 4,
 2,
 2,
 4,
 3,
 2,
 2,
 3,
 1,
 2,
 2,
 4,
 2,
 1,
 2,
 2,
 3,
 2,
 2,
 0,
 0,
 2,
 2,
 0,
 2,
 1,
 3,
 2,
 3,
 0,
 4,
 2,
 4,
 2,
 4,
 1,
 0,
 2,
 1,
 3,
 2,
 4,
 2,
 1,
 4,
 3,
 0,
 2,
 3,
 1,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 1,
 2,
 2,
 4,
 1,
 1,
 3,
 2,
 2,
 0,
 3,
 2,
 2,
 2,
 2,
 2,
 3,
 1,
 2,
 4,
 2,
 2,
 3,
 0,
 3,
 1,
 3,
 3,
 3,
 4,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 1,


## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

### Pre-Processing

In [7]:
nltk.download('wordnet')


def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    #TODO: Implement 
    
    # Lowercase the twit message
    text = message.lower()
    
    # Replace URLs with a space in the message
    regex = re.compile(r"https?://[a-z./0-9?=]+\b")
    text = re.sub(regex, " ", text)
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    regex = re.compile(r"\$\w+")
    text = re.sub(regex, " ", text)
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    regex = re.compile(r"\@\w+")
    text = re.sub(regex, " ", text)
    
    # Replace everything not a letter with a space
    regex = re.compile(r"[^a-z]+")
    text = re.sub(regex, " ", text)
    
    # Tokenize by splitting the string on whitespace into a list of words
    tokens = text.split()

    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [wnl.lemmatize(word, pos="v") for word in tokens if len(word)>1]
    
    return tokens

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [8]:
# TODO Implement

tokenized = [preprocess(message) for message in messages]
tokenized

[['great', 'buy', 'at', 'ill', 'wait'],
 [],
 ['staanalystalert',
  'for',
  'jefferies',
  'maintain',
  'with',
  'rat',
  'of',
  'hold',
  'set',
  'target',
  'price',
  'at',
  'usd',
  'our',
  'own',
  'verdict',
  'be',
  'buy'],
 ['hear',
  'there',
  'guy',
  'who',
  'know',
  'someone',
  'who',
  'think',
  'somebody',
  'know',
  'something',
  'on',
  'stocktwits'],
 ['reveal', 'yourself'],
 ['why', 'the', 'drop', 'warren', 'buffet', 'take', 'out', 'his', 'position'],
 ['bear', 'have', 'reason', 'on', 'to', 'pay', 'more', 'attention'],
 ['ok',
  'good',
  'we',
  're',
  'not',
  'drop',
  'in',
  'price',
  'over',
  'the',
  'weekend',
  'lol'],
 ['daily', 'chart', 'we', 'need', 'to', 'get', 'back', 'to', 'above'],
 ['drop',
  'per',
  'week',
  'after',
  'spike',
  'if',
  'no',
  'news',
  'in',
  'months',
  'back',
  'to',
  'if',
  'bo',
  'then',
  'bingo',
  'what',
  'be',
  'the',
  'odds'],
 ['strong', 'buy'],
 ['short', 'ratio', 'be', 'at', 'and', 'short',

### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [9]:
from collections import Counter


"""
Create a vocabulary by using Bag of words
"""

# TODO: Implement 

words = [token for tokens in tokenized for token in tokens]
#print(vocab[:10])
bow = Counter(words)
print(bow)

Counter({'be': 589347, 'the': 406384, 'to': 396297, 'amp': 393086, 'utm': 305643, 'for': 282640, 'on': 247584, 'of': 218868, 'and': 215172, 'in': 213430, 'this': 204544, 'it': 169041, 'at': 140819, 'will': 129918, 'up': 124252, 'buy': 123409, 'report': 123046, 'source': 111129, 'have': 109045, 'go': 109012, 'stocktwits': 101332, 'metric': 100343, 'you': 95348, 'short': 94452, 'medium': 92444, 'that': 90421, 'stock': 88470, 'content': 88354, 'what': 79716, 'trade': 77713, 'get': 76571, 'today': 75866, 'here': 75084, 'just': 73725, 'call': 73669, 'with': 71975, 'eps': 71607, 'sell': 71459, 'revenue': 71253, 'down': 69931, 'name': 69742, 'not': 68971, 'from': 65702, 'we': 65082, 'earn': 60670, 'all': 60536, 'do': 59910, 'if': 59262, 'now': 58446, 'market': 58137, 'growth': 57924, 'out': 57470, 'more': 56875, 'by': 56609, 'they': 56534, 'time': 55619, 'but': 54103, 'like': 53063, 'estimize': 52763, 'share': 52267, 'see': 51768, 'my': 51463, 'look': 50673, 'day': 49570, 'new': 49241, 'as': 

### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the number of messages. Then remove words that only appear in some small fraction of the messages.

In [10]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""

# TODO Implement 

# Dictionary that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
total_count = len(words)
freqs = {word: count/total_count for word, count in bow.items()}

# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 0.000007

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 14 # starting at "up"

# The k most common words in the corpus. Use `high_cutoff` as the k.
K_most_common = [word for word, count in bow.most_common(high_cutoff)]

filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]
print(K_most_common)
len(filtered_words) 

['be', 'the', 'to', 'amp', 'utm', 'for', 'on', 'of', 'and', 'in', 'this', 'it', 'at', 'will']


5169

In [11]:
filtered_words

['great',
 'buy',
 'ill',
 'wait',
 'staanalystalert',
 'jefferies',
 'maintain',
 'with',
 'rat',
 'hold',
 'set',
 'target',
 'price',
 'usd',
 'our',
 'own',
 'verdict',
 'hear',
 'there',
 'guy',
 'who',
 'know',
 'someone',
 'think',
 'somebody',
 'something',
 'stocktwits',
 'reveal',
 'yourself',
 'why',
 'drop',
 'warren',
 'buffet',
 'take',
 'out',
 'his',
 'position',
 'bear',
 'have',
 'reason',
 'pay',
 'more',
 'attention',
 'ok',
 'good',
 'we',
 're',
 'not',
 'over',
 'weekend',
 'lol',
 'daily',
 'chart',
 'need',
 'get',
 'back',
 'above',
 'per',
 'week',
 'after',
 'spike',
 'if',
 'no',
 'news',
 'months',
 'bo',
 'then',
 'bingo',
 'what',
 'odds',
 'strong',
 'short',
 'ratio',
 'float',
 'via',
 'squeeze',
 'perfect',
 'place',
 'an',
 'option',
 'straddle',
 'near',
 'support',
 'trend',
 'start',
 'new',
 'monday',
 'expect',
 'volume',
 'across',
 'key',
 'company',
 'sectors',
 'breakout',
 'strategy',
 'current',
 'portfolio',
 'bull',
 'catalysts',
 'cont

### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [12]:
vocab = {word: i for i, word in enumerate(filtered_words, 1)}
vocab

{'great': 1,
 'buy': 2,
 'ill': 3,
 'wait': 4,
 'staanalystalert': 5,
 'jefferies': 6,
 'maintain': 7,
 'with': 8,
 'rat': 9,
 'hold': 10,
 'set': 11,
 'target': 12,
 'price': 13,
 'usd': 14,
 'our': 15,
 'own': 16,
 'verdict': 17,
 'hear': 18,
 'there': 19,
 'guy': 20,
 'who': 21,
 'know': 22,
 'someone': 23,
 'think': 24,
 'somebody': 25,
 'something': 26,
 'stocktwits': 27,
 'reveal': 28,
 'yourself': 29,
 'why': 30,
 'drop': 31,
 'warren': 32,
 'buffet': 33,
 'take': 34,
 'out': 35,
 'his': 36,
 'position': 37,
 'bear': 38,
 'have': 39,
 'reason': 40,
 'pay': 41,
 'more': 42,
 'attention': 43,
 'ok': 44,
 'good': 45,
 'we': 46,
 're': 47,
 'not': 48,
 'over': 49,
 'weekend': 50,
 'lol': 51,
 'daily': 52,
 'chart': 53,
 'need': 54,
 'get': 55,
 'back': 56,
 'above': 57,
 'per': 58,
 'week': 59,
 'after': 60,
 'spike': 61,
 'if': 62,
 'no': 63,
 'news': 64,
 'months': 65,
 'bo': 66,
 'then': 67,
 'bingo': 68,
 'what': 69,
 'odds': 70,
 'strong': 71,
 'short': 72,
 'ratio': 73,
 'floa

In [13]:
len(vocab)

5169

In [14]:
id2vocab = {i: word for word, i in vocab.items()}
id2vocab

{1: 'great',
 2: 'buy',
 3: 'ill',
 4: 'wait',
 5: 'staanalystalert',
 6: 'jefferies',
 7: 'maintain',
 8: 'with',
 9: 'rat',
 10: 'hold',
 11: 'set',
 12: 'target',
 13: 'price',
 14: 'usd',
 15: 'our',
 16: 'own',
 17: 'verdict',
 18: 'hear',
 19: 'there',
 20: 'guy',
 21: 'who',
 22: 'know',
 23: 'someone',
 24: 'think',
 25: 'somebody',
 26: 'something',
 27: 'stocktwits',
 28: 'reveal',
 29: 'yourself',
 30: 'why',
 31: 'drop',
 32: 'warren',
 33: 'buffet',
 34: 'take',
 35: 'out',
 36: 'his',
 37: 'position',
 38: 'bear',
 39: 'have',
 40: 'reason',
 41: 'pay',
 42: 'more',
 43: 'attention',
 44: 'ok',
 45: 'good',
 46: 'we',
 47: 're',
 48: 'not',
 49: 'over',
 50: 'weekend',
 51: 'lol',
 52: 'daily',
 53: 'chart',
 54: 'need',
 55: 'get',
 56: 'back',
 57: 'above',
 58: 'per',
 59: 'week',
 60: 'after',
 61: 'spike',
 62: 'if',
 63: 'no',
 64: 'news',
 65: 'months',
 66: 'bo',
 67: 'then',
 68: 'bingo',
 69: 'what',
 70: 'odds',
 71: 'strong',
 72: 'short',
 73: 'ratio',
 74: '

In [15]:
filtered = [[token for token in message if token in vocab] for message in tokenized]
filtered

[['great', 'buy', 'ill', 'wait'],
 [],
 ['staanalystalert',
  'jefferies',
  'maintain',
  'with',
  'rat',
  'hold',
  'set',
  'target',
  'price',
  'usd',
  'our',
  'own',
  'verdict',
  'buy'],
 ['hear',
  'there',
  'guy',
  'who',
  'know',
  'someone',
  'who',
  'think',
  'somebody',
  'know',
  'something',
  'stocktwits'],
 ['reveal', 'yourself'],
 ['why', 'drop', 'warren', 'buffet', 'take', 'out', 'his', 'position'],
 ['bear', 'have', 'reason', 'pay', 'more', 'attention'],
 ['ok', 'good', 'we', 're', 'not', 'drop', 'price', 'over', 'weekend', 'lol'],
 ['daily', 'chart', 'we', 'need', 'get', 'back', 'above'],
 ['drop',
  'per',
  'week',
  'after',
  'spike',
  'if',
  'no',
  'news',
  'months',
  'back',
  'if',
  'bo',
  'then',
  'bingo',
  'what',
  'odds'],
 ['strong', 'buy'],
 ['short', 'ratio', 'short', 'float', 'via'],
 ['price',
  'squeeze',
  'perfect',
  'place',
  'an',
  'option',
  'straddle',
  'near',
  'support',
  'trend'],
 ['start',
  'new',
  'monday'

### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [16]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [17]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.1953718386637769

Finally let's convert our tokens into integer ids which we can pass to the network.

In [18]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']

In [19]:
token_ids

[[1, 2, 3, 4],
 [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 2],
 [18, 19, 20, 21, 22, 23, 21, 24, 25, 22, 26, 27],
 [30, 31, 32, 33, 34, 35, 36, 37],
 [38, 39, 40, 41, 42, 43],
 [44, 45, 46, 47, 48, 31, 13, 49, 50, 51],
 [52, 53, 46, 54, 55, 56, 57],
 [31, 58, 59, 60, 61, 62, 63, 64, 65, 56, 62, 66, 67, 68, 69, 70],
 [71, 2],
 [72, 73, 72, 74, 75],
 [13, 76, 77, 78, 79, 80, 81, 82, 83, 84],
 [85, 86, 87, 88, 71, 2, 89, 90, 91, 92, 93],
 [94, 95, 96, 97],
 [98, 39, 40, 41, 42, 43],
 [99, 100, 86, 101, 102, 103, 2, 35, 104, 105, 106, 107, 108, 109, 110],
 [5, 115, 116, 11, 13, 12, 8, 9, 2, 11, 12, 13, 14, 15, 16, 17, 2],
 [24,
  117,
  118,
  119,
  69,
  120,
  121,
  87,
  122,
  8,
  96,
  64,
  123,
  124,
  19,
  125,
  126,
  127],
 [39,
  96,
  73,
  126,
  128,
  129,
  39,
  63,
  130,
  131,
  132,
  133,
  134,
  27,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  136,
  91,
  143],
 [144, 145, 146, 147, 144, 145, 148, 149, 150, 151, 152, 153],
 [154, 72],
 [155,

In [20]:
sentiments

[4,
 4,
 3,
 3,
 0,
 3,
 4,
 0,
 4,
 0,
 4,
 4,
 4,
 3,
 4,
 4,
 3,
 3,
 3,
 0,
 1,
 3,
 3,
 0,
 4,
 2,
 4,
 0,
 3,
 1,
 3,
 3,
 4,
 3,
 2,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 1,
 4,
 3,
 4,
 0,
 3,
 2,
 0,
 1,
 4,
 1,
 3,
 4,
 1,
 0,
 4,
 2,
 4,
 4,
 3,
 4,
 2,
 0,
 0,
 0,
 3,
 1,
 1,
 2,
 2,
 3,
 0,
 0,
 4,
 4,
 2,
 2,
 3,
 2,
 3,
 4,
 0,
 4,
 3,
 3,
 4,
 4,
 1,
 1,
 4,
 2,
 2,
 3,
 2,
 2,
 4,
 1,
 2,
 0,
 3,
 0,
 4,
 4,
 3,
 2,
 3,
 1,
 2,
 4,
 1,
 3,
 2,
 0,
 0,
 0,
 1,
 3,
 3,
 0,
 4,
 4,
 4,
 1,
 0,
 2,
 1,
 3,
 2,
 4,
 2,
 1,
 4,
 3,
 0,
 3,
 1,
 3,
 2,
 3,
 1,
 2,
 4,
 1,
 1,
 3,
 0,
 3,
 2,
 2,
 3,
 1,
 4,
 2,
 3,
 0,
 3,
 1,
 3,
 3,
 3,
 4,
 2,
 2,
 2,
 0,
 2,
 1,
 0,
 0,
 0,
 0,
 3,
 0,
 1,
 3,
 4,
 3,
 2,
 3,
 1,
 1,
 2,
 4,
 3,
 2,
 3,
 3,
 1,
 1,
 1,
 0,
 3,
 4,
 2,
 4,
 0,
 3,
 0,
 4,
 4,
 3,
 3,
 2,
 3,
 0,
 0,
 2,
 2,
 0,
 1,
 2,
 3,
 1,
 1,
 2,
 3,
 3,
 1,
 3,
 0,
 1,
 4,
 3,
 2,
 0,
 0,
 3,
 3,
 4,
 3,
 1,
 3,
 4,
 0,
 2,
 3,
 4,
 4,
 3,
 4,
 3,
 3,
 1,
 2,
 2,
 1,
 4,
 2,


## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [21]:
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [22]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout
        
        # TODO Implement

        # Setup embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, lstm_size, lstm_layers, dropout=dropout, batch_first=False)
        
        # Setup additional layers
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(lstm_size, output_size)
        self.log_soft = nn.LogSoftmax(dim=1)

    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        
        # TODO Implement 
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        
        weight = next(self.parameters()).data
        #hidden = ()
        
        hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                      weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())
            
        return hidden


    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        
        # TODO Implement 
        
        batch_size = nn_input.size(1)
        nn_input = nn_input.long()
        embeds = self.embedding(nn_input)
        lstm_out, hidden_state = self.lstm(embeds, hidden_state)
        lstm_out = lstm_out[-1, :, :]
        out = self.dropout(lstm_out)
        out = self.fc(out)
        log_prob = self.log_soft(out)
        
        return log_prob, hidden_state

### View Model

In [23]:
model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=2)
model.embedding.weight.data.uniform_(-1, 1)
input = torch.randint(0, 1000, (5, 4), dtype=torch.int64)
hidden = model.init_hidden(4)

logps, _ = model.forward(input, hidden)
print(logps)
print(model)

tensor([[-1.7502, -1.5774, -1.8203, -1.4170, -1.5356],
        [-1.7248, -1.6491, -1.7325, -1.4818, -1.4895],
        [-1.7612, -1.5833, -1.8354, -1.4112, -1.5165],
        [-1.7690, -1.6346, -1.7964, -1.3957, -1.5100]])
TextClassifier(
  (embedding): Embedding(5169, 10)
  (lstm): LSTM(10, 6, num_layers=2, dropout=0.1)
  (dropout): Dropout(p=0.1)
  (fc): Linear(in_features=6, out_features=5, bias=True)
  (log_soft): LogSoftmax()
)


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [24]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [25]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   

# TODO Implement 

split_frac = 0.8
split_idx = int(len(token_ids)*split_frac)

train_features = token_ids[:split_idx]
valid_features = token_ids[split_idx:]
train_labels = sentiments[:split_idx]
valid_labels = sentiments[split_idx:]

In [26]:
print(len(train_features)/len(token_ids))
print(len(train_features))

0.7999998061421908
825347


In [27]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)
hidden = model.init_hidden(64)
logps, hidden = model.forward(text_batch, hidden)

In [28]:
ps = torch.exp(logps)
top_p, top_class = ps.topk(1, dim=1)
print(top_p.shape)
print(top_class.shape)

torch.Size([64, 1])
torch.Size([64, 1])


### Training
It's time to train the neural network!

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab)+1, 1024, 512, 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

TextClassifier(
  (embedding): Embedding(5170, 1024)
  (lstm): LSTM(1024, 512, num_layers=2, dropout=0.2)
  (dropout): Dropout(p=0.2)
  (fc): Linear(in_features=512, out_features=5, bias=True)
  (log_soft): LogSoftmax()
)

In [30]:
"""
Train your model with dropout. Make sure to clip your gradients.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""

epochs = 3
batch_size = 200
learning_rate = 0.003

print_every = 100
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.train()

for epoch in range(epochs):
    print('Starting epoch {}'.format(epoch + 1))
    steps = 0
    
    for text_batch, labels in dataloader(
            train_features, train_labels, batch_size=batch_size, sequence_length=20, shuffle=True):
        
        steps += 1
        hidden = model.init_hidden(labels.shape[0])
        
        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)
        for each in hidden:
            each.to(device)
        
        # TODO Implement: Train Model
        optimizer.zero_grad()
        output, hidden = model(text_batch, hidden)
        loss = criterion(output, labels)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 5)
        optimizer.step()
        
        if steps % print_every == 0:
            model.eval()
            valid_losses = []
            accuracy = []
            # TODO Implement: Print metrics
            for inputs, labels in dataloader(
                    valid_features, valid_labels, batch_size=batch_size, sequence_length=20, shuffle=True):
                
                valid_hidden = model.init_hidden(labels.shape[0])
                inputs, labels = inputs.to(device), labels.to(device)
                
                for each in valid_hidden:
                    each.to(device)
                    
                valid_output, valid_hidden = model(inputs, valid_hidden)
                valid_loss = criterion(valid_output, labels)
                valid_losses.append(valid_loss.item())
                ps = torch.exp(valid_output)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy.append(torch.mean(equals.type(torch.FloatTensor)).item())
                                            
            model.train()
            
            print("Epoch: {}/{}...".format(epoch+1, epochs),
                  "Step: {}...".format(steps),
                  "Training Loss: {:.6f}...".format(loss.item()),
                  "Validation Loss: {:.6f}".format(sum(valid_losses)/len(valid_losses)),
                  "Validation Accuracy: {:.6f}...".format(sum(accuracy)/len(accuracy))
                 )

Starting epoch 1
Epoch: 1/3... Step: 100... Training Loss: 0.985253... Validation Loss: 1.015304 Validation Accuracy: 0.598363...
Epoch: 1/3... Step: 200... Training Loss: 0.924843... Validation Loss: 0.925857 Validation Accuracy: 0.634391...
Epoch: 1/3... Step: 300... Training Loss: 0.920315... Validation Loss: 0.889260 Validation Accuracy: 0.659117...
Epoch: 1/3... Step: 400... Training Loss: 0.857503... Validation Loss: 0.857864 Validation Accuracy: 0.670401...
Epoch: 1/3... Step: 500... Training Loss: 0.879947... Validation Loss: 0.833633 Validation Accuracy: 0.673935...
Epoch: 1/3... Step: 600... Training Loss: 0.808672... Validation Loss: 0.831748 Validation Accuracy: 0.677329...
Epoch: 1/3... Step: 700... Training Loss: 0.797211... Validation Loss: 0.809537 Validation Accuracy: 0.690901...
Epoch: 1/3... Step: 800... Training Loss: 0.751977... Validation Loss: 0.807284 Validation Accuracy: 0.687694...
Epoch: 1/3... Step: 900... Training Loss: 0.832557... Validation Loss: 0.799320

Epoch: 2/3... Step: 3200... Training Loss: 0.797814... Validation Loss: 0.740513 Validation Accuracy: 0.718303...
Epoch: 2/3... Step: 3300... Training Loss: 0.679923... Validation Loss: 0.740728 Validation Accuracy: 0.717905...
Epoch: 2/3... Step: 3400... Training Loss: 0.696336... Validation Loss: 0.741133 Validation Accuracy: 0.717794...
Epoch: 2/3... Step: 3500... Training Loss: 0.823443... Validation Loss: 0.735387 Validation Accuracy: 0.718196...
Epoch: 2/3... Step: 3600... Training Loss: 0.809271... Validation Loss: 0.741402 Validation Accuracy: 0.718066...
Epoch: 2/3... Step: 3700... Training Loss: 0.632331... Validation Loss: 0.740163 Validation Accuracy: 0.716737...
Epoch: 2/3... Step: 3800... Training Loss: 0.671546... Validation Loss: 0.739382 Validation Accuracy: 0.717898...
Epoch: 2/3... Step: 3900... Training Loss: 0.681003... Validation Loss: 0.737635 Validation Accuracy: 0.720297...
Epoch: 2/3... Step: 4000... Training Loss: 0.743095... Validation Loss: 0.737763 Validat

## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [38]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    
    # TODO Implement
    
    tokens = preprocess(text)
    
    # Filter non-vocab words
    tokens = [word for word in tokens if word in vocab]
    # Convert words to ids
    tokens = [vocab[word] for word in tokens]
        
    # Adding a batch dimension
    text_input = torch.FloatTensor(tokens).view(-1,1)
    text_input = torch.from_numpy(np.asarray(text_input))
    # Get the NN output
    hidden = model.init_hidden(1)
    logps, _ = model(text_input, hidden)
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = torch.exp(logps)
    
    return pred

In [39]:
text = "Google is working on self driving cars, I'm bullish on $goog"
model.eval()
model.to("cpu")
predict(text, model, vocab)

tensor([[ 0.0005,  0.0094,  0.0126,  0.8569,  0.1206]])

In [41]:
1-0.8569

0.1431

### Questions: What is the prediction of the model? What is the uncertainty of the prediction?
** TODO: Answer Question**

The text classifier model predicts that the given text carries positive sentiment, since the fourth class carries the highest probability (85.69%). The uncertainity of this prediction is 14.31%. It seems like the model is predicting the correct sentiment. Furthermore, the progression of certainty from very negative to positive and from positive to very positive seem to be logical (very certain the twit does not carry very negative sentiment, certain that the twit does not carry negative or neutral sentiment, very certain that the twit carries positive sentiment, and slightly uncertain about whether the twit may even be very positive). The training loss fluctuated a lot between 0.7 and 0.8, whereas the validation loss appears to have remained relatively constant around 0.75 after the first epoch finished (same accounts for validation accuracy). Maybe there is some overfitting going on after the first epoch, so decreasing the number of epochs, or including early stopping or cross-validation would improve the accuracy of the model whilst minimizing overfitting.

Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [42]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [43]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

{'message_body': '$JWN has moved -1.69% on 10-31. Check out the movement and peers at  https://dividendbot.com?s=JWN',
 'timestamp': '2018-11-01T00:00:05Z'}

Using the `prediction` function, let's apply it to a stream of twits.

In [44]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [45]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

{'symbol': '$AAPL',
 'score': tensor([[ 0.1175,  0.1139,  0.3221,  0.3184,  0.1281]]),
 'timestamp': '2018-11-01T00:00:18Z'}

That's it. You have successfully built a model for sentiment analysis! 

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.